<a href="https://colab.research.google.com/github/ssenichev/cs102new/blob/homework06/homework06/SQL_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import sqlite3

df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/music_genre.csv')
con = sqlite3.connect(":memory:")
df.to_sql("music_genre", con=con)
cur = con.cursor()
df.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


### Задание 1

In [11]:
cur.execute("SELECT artist_name FROM music_genre WHERE artist_name = 'empty_field'")
cur.execute("DELETE FROM music_genre WHERE artist_name = 'empty_field'")
cur.execute(
    """DELETE FROM music_genre
WHERE artist_name IS NULL;"""
)

### Задание 2

In [12]:
cur.execute(
    "SELECT music_genre, COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS genre_percentage FROM music_genre GROUP BY music_genre;"
)
cur.fetchall()

[('Alternative', 9.951379680495043),
 ('Anime', 9.951379680495043),
 ('Blues', 9.987160868009514),
 ('Classical', 9.964008334911915),
 ('Country', 10.058723243038454),
 ('Electronic', 10.054513691566164),
 ('Hip-Hop', 10.008208625370967),
 ('Jazz', 10.039780261413146),
 ('Rap', 9.970322662120351),
 ('Rock', 10.014522952579403)]

In [8]:
cur.execute(
    "SELECT music_genre, danceability FROM music_genre WHERE danceability = (SELECT MAX(danceability) FROM music_genre)"
)
cur.fetchall()

[('Hip-Hop', 0.986)]

In [9]:
cur.execute(
    "SELECT music_genre, mode, COUNT(*) AS genre_mode "
    "FROM music_genre "
    "GROUP BY music_genre, mode "
    "ORDER BY music_genre, genre_mode DESC;"
)
cur.fetchall()

[('Alternative', 'Major', 3000),
 ('Alternative', 'Minor', 1728),
 ('Anime', 'Major', 2915),
 ('Anime', 'Minor', 1813),
 ('Blues', 'Major', 3379),
 ('Blues', 'Minor', 1366),
 ('Classical', 'Major', 3147),
 ('Classical', 'Minor', 1587),
 ('Country', 'Major', 4164),
 ('Country', 'Minor', 615),
 ('Electronic', 'Major', 2571),
 ('Electronic', 'Minor', 2206),
 ('Hip-Hop', 'Major', 2635),
 ('Hip-Hop', 'Minor', 2120),
 ('Jazz', 'Major', 2527),
 ('Jazz', 'Minor', 2243),
 ('Rap', 'Major', 2626),
 ('Rap', 'Minor', 2111),
 ('Rock', 'Major', 3521),
 ('Rock', 'Minor', 1237)]

### Задание 3

In [13]:
cur.execute("SELECT artist_name, COUNT(*) AS Value FROM music_genre GROUP BY artist_name ORDER BY Value LIMIT 1")
cur.fetchall()

[('"Weird Al" Yankovic', 1)]

In [16]:
cur.execute("SELECT artist_name, COUNT(*) AS Value FROM music_genre GROUP BY artist_name ORDER BY Value DESC LIMIT 1")
cur.fetchall()

[('Nobuo Uematsu', 429)]

In [17]:
cur.execute("SELECT artist_name, COUNT(*) AS Value FROM music_genre GROUP BY artist_name ORDER BY Value DESC LIMIT 20")
cur.fetchall()

[('Nobuo Uematsu', 429),
 ('Wolfgang Amadeus Mozart', 402),
 ('Ludwig van Beethoven', 317),
 ('Johann Sebastian Bach', 314),
 ('Frédéric Chopin', 241),
 ('Drake', 172),
 ('Capcom Sound Team', 169),
 ('Yuki Hayashi', 167),
 ('Eminem', 147),
 ('Future', 124),
 ('Thievery Corporation', 122),
 ('The Black Keys', 114),
 ('Pyotr Ilyich Tchaikovsky', 103),
 ('Kevin Gates', 102),
 ('Mac Miller', 97),
 ('Howard Shore', 93),
 ('Logic', 92),
 ('$uicideBoy$', 92),
 ('Ryuichi Sakamoto', 90),
 ('ASIAN KUNG-FU GENERATION', 89)]

### Задание 8

In [ ]:
cur.execute("ALTER TABLE music_genre ADD COLUMN track_length")

In [18]:
cur.execute(
    """
    SELECT track_length, COUNT(*) AS track_count
    FROM (
        SELECT
            CASE
                WHEN (duration_ms * 0.001 / 60) <= 3 THEN 'короткая'
                WHEN (duration_ms * 0.001 / 60) > 5 THEN 'длинная'
                WHEN 3 < (duration_ms * 0.001 / 60) <= 5 THEN 'средняя'
            END AS track_length
        FROM music_genre
    ) t
    GROUP BY track_length
    ORDER BY track_count DESC;
"""
)
cur.fetchall()

[('средняя', 27009), ('короткая', 12924), ('длинная', 7578)]